## Домашнее задание 5

**Задание 1.** Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные векторайзеры добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы


### Решение:

Установим и импортируем необходимые библиотеки.

In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import nltk
nltk.download('tagsets')

from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

import pyconll

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


Загрузим датасет и соберем обучающую и тестовую выборки.

In [3]:
!mkdir dataset_ru

!wget -O ./dataset_ru/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget -O ./dataset_ru/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-08-27 18:42:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’

./dataset_ru/ru_syn 100%[===================>]  38.85M   190MB/s    in 0.2s    

2022-08-27 18:42:20 (190 MB/s) - ‘./dataset_ru/ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-08-27 18:42:20--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [5]:
full_train = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-train-c.conllu')

# Общая обучающая выборка
full_train.extend([*full_train_b, *full_train_c])

full_test = pyconll.load_from_file('dataset_ru/ru_syntagrus-ud-dev.conllu')

Пример текста из тестового датасета с соответсвующими тэгами.

In [6]:
for sent in full_test[:1]:
    for token in sent:
        print(token.form, token.upos)
    print()

Алгоритм NOUN
, PUNCT
от ADP
имени NOUN
учёного NOUN
аль PART
- PUNCT
Хорезми PROPN
, PUNCT
- PUNCT
точный ADJ
набор NOUN
инструкций NOUN
, PUNCT
описывающих VERB
порядок NOUN
действий NOUN
исполнителя NOUN
для ADP
достижения NOUN
результата NOUN
решения NOUN
задачи NOUN
за ADP
конечное ADJ
время NOUN
. PUNCT



Преобразовываем выборки для удобства дальнейшего использования при обучении и тестировании.

In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

Попробуем разные стандартные методы для определения тэгов слов.

In [8]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[50]), unigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('нём', 'PRON'),
 ('алгорифм', None),
 ('(', 'PUNCT'),
 ('кстати', 'ADV'),
 (',', 'PUNCT'),
 ('до', 'ADP'),
 ('революции', 'NOUN'),
 ('использовалось', 'VERB'),
 ('написание', 'NOUN'),
 ('алгорифм', None),
 (',', 'PUNCT'),
 ('через', 'ADP'),
 ('фиту', None),
 (')', 'PUNCT'),
 ('производится', 'VERB'),
 ('"', 'PUNCT'),
 ('от', 'ADP'),
 ('арабского', 'ADJ'),
 ('слова', 'NOUN'),
 ('Аль-Горетм', None),
 (',', 'PUNCT'),
 ('то', 'SCONJ'),
 ('есть', 'VERB'),
 ('корень', 'NOUN'),
 ('"', 'PUNCT'),
 ('.', 'PUNCT')]

0.8782863467673677

In [9]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(fdata_sent_test[50]), bigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('нём', 'PRON'),
 ('алгорифм', None),
 ('(', 'PUNCT'),
 ('кстати', 'ADV'),
 (',', 'PUNCT'),
 ('до', 'ADP'),
 ('революции', 'NOUN'),
 ('использовалось', None),
 ('написание', None),
 ('алгорифм', None),
 (',', 'PUNCT'),
 ('через', 'ADP'),
 ('фиту', None),
 (')', 'PUNCT'),
 ('производится', None),
 ('"', 'PUNCT'),
 ('от', 'ADP'),
 ('арабского', 'ADJ'),
 ('слова', 'NOUN'),
 ('Аль-Горетм', None),
 (',', 'PUNCT'),
 ('то', 'SCONJ'),
 ('есть', 'VERB'),
 ('корень', None),
 ('"', 'PUNCT'),
 ('.', 'PUNCT')]

0.7101308678950452

In [10]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(fdata_sent_test[50]), trigram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('нём', None),
 ('алгорифм', None),
 ('(', None),
 ('кстати', None),
 (',', 'PUNCT'),
 ('до', 'ADP'),
 ('революции', 'NOUN'),
 ('использовалось', None),
 ('написание', None),
 ('алгорифм', None),
 (',', 'PUNCT'),
 ('через', None),
 ('фиту', None),
 (')', None),
 ('производится', None),
 ('"', 'PUNCT'),
 ('от', None),
 ('арабского', None),
 ('слова', None),
 ('Аль-Горетм', None),
 (',', 'PUNCT'),
 ('то', None),
 ('есть', None),
 ('корень', None),
 ('"', 'PUNCT'),
 ('.', 'PUNCT')]

0.4067191874470994

Как видим самые лучшие результаты показывает UnigramTagger, который не использует соседние слова для определения тэга слова.

Попробуем создать комбинацию тэггеров, чтобы уменьшить количество нераспознаваемых слов, и посмотрим, как это повлияет на точность модели.

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


# В качестве бэкофф опции будем использовать тэг существительного
backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.9119799466111075

Как видим благодаря этому подходу удалось добиться улучшения точности модели.

Теперь построим модель тэггера на основе разных векторайзеров, которая будет фактически классифицировать каждое слово из датасета. Для этого для начала подготовим обучающий и тестовый наборы данных. 

In [13]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [14]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [15]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'],
      dtype='<U5')

Создадим несколько векторайзеров и обучим модель на их основе. Затем сравним получившиеся результаты.

In [18]:
hvectorizer = HashingVectorizer(ngram_range=(2, 15), analyzer='char', n_features=65536)
tvectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word')
cvectorizer = CountVectorizer(ngram_range=(2, 13), analyzer='char')

In [19]:
Xh_train = hvectorizer.fit_transform(train_tok)
Xh_test = hvectorizer.transform(test_tok)

Xt_train = tvectorizer.fit_transform(train_tok)
Xt_test = tvectorizer.transform(test_tok)

Xc_train = cvectorizer.fit_transform(train_tok)
Xc_test = cvectorizer.transform(test_tok)

In [20]:
%%time
lr = LogisticRegression(random_state=42, max_iter=500)
lr.fit(Xh_train, train_enc_labels)
pred = lr.predict(Xh_test)
print(f'Accuracy для модели тэггера на основе HashingVectorizer - {accuracy_score(test_enc_labels, pred):.4f}.')

Accuracy для модели тэггера на основе HashingVectorizer - 0.8609.
CPU times: user 22min 9s, sys: 2min 50s, total: 24min 59s
Wall time: 21min 55s


In [21]:
%%time
lr = LogisticRegression(random_state=42, max_iter=500)
lr.fit(Xt_train, train_enc_labels)
pred = lr.predict(Xt_test)
print(f'Accuracy для модели тэггера на основе TfidfVectorizer - {accuracy_score(test_enc_labels, pred):.4f}.')

Accuracy для модели тэггера на основе TfidfVectorizer - 0.7563.
CPU times: user 16min 12s, sys: 3min 2s, total: 19min 14s
Wall time: 15min 38s


In [22]:
%%time
lr = LogisticRegression(random_state=42, max_iter=500)
lr.fit(Xc_train, train_enc_labels)
pred = lr.predict(Xc_test)
print(f'Accuracy для модели тэггера на основе CountVectorizer - {accuracy_score(test_enc_labels, pred):.4f}.')

Accuracy для модели тэггера на основе CountVectorizer - 0.8695.
CPU times: user 53min 7s, sys: 9min, total: 1h 2min 8s
Wall time: 53min 36s


Как видим наилучшая точность получилась среди векторайзеров получилась для модели CountVectorizer на основе букв - 0.8695. Тем не менее комбинация стандартных тэггеров рассмотренная ранее позволила добиться еще большей точности в определении тэгов слов - 0.9120. 